<a href="https://colab.research.google.com/github/halfbug/colab/blob/main/workroom_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation and Configuration

In [ ]:
import pandas as pd
try:
  import openai
  from langchain_openai import ChatOpenAI
except:
    !pip install openai
    !pip install langchain
    !pip install langchain-openai
    # !pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [ ]:
import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.chat_models import ChatOpenAI
import datetime
current_date = datetime.datetime.now().date()
llm_name = "gpt-4"
# if current_date < datetime.date(2023, 9, 2):
#     llm_name = "gpt-3.5-turbo-0301"
# else:
#     llm_name = "gpt-3.5-turbo"
print(llm_name)
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.invoke("Hello world!")

gpt-4


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e36f7f0-08b3-44f3-8595-948fa82812bd-0')

In [ ]:
from langchain.tools import tool
import requests
from pydantic import BaseModel, Field
from langchain_core.pydantic_v1 import BaseModel

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


In [ ]:
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.agent import AgentFinish
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions

# ExcelProcessor

In [ ]:
class ExcelProcessor:
    def __init__(self, file_path=None, sheet_id=None, sheet_name="Excel file"):
        # Example usage for opening a Google Sheet:
        # sheet_id = '1M8mauGQDZVbXpwQPAlfrKc2suVqtdB5k'
        # sheet_name = "LevelGuide"
        # processor = ExcelProcessor(sheet_id=sheet_id, sheet_name=sheet_name)

        if file_path:
            self.df = pd.read_excel(file_path)
        elif sheet_id and sheet_name:
            url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
            self.df = pd.read_csv(url)
        else:
            raise ValueError("Either provide a file path or Google Sheet ID and sheet name.")
        # self.logger = Logger(__name__).logger

    def get_empty_columns_names(self):
        empty_columns = self.df.columns[self.df.isnull().any()].tolist()
        return empty_columns

    def get_filled_columns_names(self):
        filled_columns = self.df.columns[self.df.notnull().any()].tolist()
        return filled_columns

    def list_columns_with_values(self,num_columns=None):
        non_empty_columns = self.df.columns[self.df.notnull().any()].tolist()
        filtered_df = self.df[non_empty_columns]
        if num_columns:
            return filtered_df.values[:, :num_columns].tolist()
        else:
            return filtered_df.values.tolist()
        return columns_with_values

    def get_all_rows(self):
        return self.df.values.tolist()

    def get_all_rows_values(self):
        non_empty_columns = self.df.columns[self.df.notnull().any()].tolist()
        filtered_df = self.df[non_empty_columns]
        return filtered_df.values.tolist()

    def get_total_rows_columns(self):
        total_rows, total_columns = self.df.shape
        self.logger.info("get total rows and columns in the sheet")
        return total_rows, total_columns
        # Example usage to get total rows and columns in the sheet
        # total_rows, total_columns = processor.get_total_rows_columns()
        # print("Total rows:", total_rows)
        # print("Total columns:", total_columns)

# HelperText

In [ ]:
class HelperText:
    @staticmethod
    def format_string(input_string):
        formatted_string = ""

        # Split the input string into separate lines
        lines = input_string.split('\n')

        # Iterate through each line
        for line in lines:
            # Check if the line contains a colon ':'
            if ':' in line:
                # Split the line into key and value based on the colon ':'
                key, value = line.split(':', 1)
                # Format the key and value and add them to the formatted string
                formatted_string += f"{key.strip()}: {value.strip()} "
            else:
                # If the line does not contain a colon, add it as is to the formatted string
                formatted_string += f"{line.strip()} "

        return formatted_string


    @staticmethod
    def clear_code(input_string):
        string_encode = input_string.encode("ascii", "ignore")
        input_string = string_encode.decode()
        return(input_string)

# Example usage:
helper = HelperText()

# Format a string
formatted_string = helper.format_string("Your input string")
print("Formatted string:", formatted_string)

# Clear code from a string
cleaned_string = helper.clear_code("Your input string with code")
print("Cleaned string:", cleaned_string)


Formatted string: Your input string 
Cleaned string: Your input string with code


# WebScraper

In [ ]:
from googleapiclient.discovery import build
import re
import requests
from bs4 import BeautifulSoup
from google.colab import userdata

class WebScraper:
    def __init__(self):
        self.base_url = "https://www.google.com/search?q="
        self.api_key = userdata.get('GOOGLE_SEARCH_API')
        self.service = build("customsearch", "v1", developerKey=self.api_key)
        # self.logger = Logger(__name__).logger

    def google_scraper(self, query):
        try:
          response = requests.get(self.base_url+query.lower().replace(' ', '+'))
          soup = BeautifulSoup(response.content, 'html.parser')

          # find first five link's content
          # anchor_tags = soup.find_all('a', href=True)
          # hrefs = []
          # content = ""
          # for tag in anchor_tags:
          #   href = tag.get('href')
          #   if href and href.startswith("/url?q=https") and "google" not in href.lower():  # Check if href exists (it might be None)
          #     parts = href.split('&', 1)
          #     parts = parts[0].split('/url?q=',1)
          #     # print(parts)
          #     hrefs.append(parts[1])
          #     scraped_content = scrape_text(parts[1])
          #     content += scraped_content + "\n ***"
              # print(scraped_content)
          # print(hrefs[:5])



          body_content = soup.find('body')  # Find the body tag

          if body_content:
            # Extract only the text content from the body tag
              text_content = body_content.get_text(strip=True)
              # print("\033[34m" + text_content )



              return helper.clear_code(text_content)    #format_string(content)
        except Exception as e:
          print(f"Error occurred while scraping: {e}")
          return "No search result found: Unknown"
        self.logger.info("Scraping Google search results for: %s", query)

    def scrape_text(self,url):
    # Function to scrape the webpage for availability information
      try:
          response = requests.get(url)
          soup = BeautifulSoup(response.content, 'html.parser')
          # Search for specific keywords like "stock availability" on the webpage

          # Step 3: Extract metadata
          meta_tags = soup.find_all('meta')
          metadata = {}
          for tag in meta_tags:
              if 'name' in tag.attrs:
                  name = tag.attrs['name']
                  content = tag.attrs.get('content', '')
                  metadata[name] = content
              elif 'property' in tag.attrs:  # For OpenGraph metadata
                  property = tag.attrs['property']
                  content = tag.attrs.get('content', '')
                  metadata[property] = content

          # Display the metadata
          # for key, value in metadata.items():
          #     print(f"{key}: {value}")

          # print( metadata)

          meta_tag = self.extract_product_info([metadata])

              # Find all link elements with the 'as' attribute set to 'image'
          image_links = soup.find_all('link', {'rel': 'preload', 'as': 'image'})

          # Extract and print the href attributes
          images = ''
          for link in image_links:
              image_url = link.get('href')
              if image_url:
                  if image_url.startswith("//"):
                    # Add protocol (http assumed):
                    image_url= f"http:{image_url}"
                  images = images +","+ image_url
                  # print(image_url)

          body_content = soup.find('body')  # Find the body tag
          print(body_content)
          if body_content:
            # Extract only the text content from the body tag
              text_content = body_content.get_text(strip=True)

              # Search for either "stock" or "stock availability" in the text content
              # print(text_content)
              string_encode = text_content.encode("ascii", "ignore")
              text_content = string_encode.decode()
              return text_content + meta_tag + "Images : " + images
          return f"Content: Unknown on {url}"
      except Exception as e:
          print(f"Error occurred while scraping: {e}")
          return f"Content: Unknown on {url}"

    def extract_product_info(self, meta_tags):
        # Function to extract product information from meta tags
        product_info = ""
        for meta_tag in meta_tags:
            if 'og:title' in meta_tag:
                product_info += f"\n ***Title: {meta_tag['og:title']}\n"
            if 'og:description' in meta_tag:
                product_info += f"Description: {meta_tag['og:description']}\n"
            if 'og:price:amount' in meta_tag and 'og:price:currency' in meta_tag:
                product_info += f"Price: {meta_tag['og:price:amount']} {meta_tag['og:price:currency']}\n"
            if 'og:image' in meta_tag:
                    if len(meta_tag['og:image']) <= 200:  # Check if URL length is less than or equal to 200
                        product_info += f"Image URL: {meta_tag['og:image']} \n"
            if 'og:url' in meta_tag:
                product_info += f"URL: {meta_tag['og:url']} \n"
                # Call function to extract availability information
                # availability_info = self.extract_availability(meta_tag['og:url'])
                # product_info += f"{availability_info}\n"
                # Extract ASIN from URL if it's an Amazon link
                asin_match = re.search(r'/dp/(\w+)', meta_tag['og:url'])
                if asin_match:
                    product_info += f"ASIN: {asin_match.group(1)} \n"
            if 'og:site_name' in meta_tag:
                product_info += f"Site Name: {meta_tag['og:site_name']} \n"
            if 'og:availability' in meta_tag:
                product_info += f"Availability: {meta_tag['og:availability']} \n"
            if 'og:type' in meta_tag:
                    product_info += f"type: {meta_tag['og:type']} \n"
            else:
              product_info += f"type: text \n"
        return product_info


    def extract_availability(self,url):
    # Function to scrape the webpage for availability information
      try:
          response = requests.get(url)
          soup = BeautifulSoup(response.content, 'html.parser')
          # Search for specific keywords like "stock availability" on the webpage

          body_content = soup.find('body')  # Find the body tag

          if body_content:
            # Extract only the text content from the body tag
              text_content = body_content.get_text(strip=True)
              # Search for either "stock" or "stock availability" in the text content
              # print(text_content)
              #  availability_pattern = re.compile(r"(?:stock|stock|availability|Number\sof\sItems\s*(\d+)|in\s*stock|(\w+)\s*stock\s*(\w+))", re.IGNORECASE)

              availability_pattern = re.compile(r"(?:stock|stock|availability|in\s*stock)", re.IGNORECASE)
              match = re.search(availability_pattern, text_content)
              # print(match.groups())
              if match:
                  # print(match.group(0))
                  return f"Availability: {match.group(0).strip()}"
          # If specific keywords not found, return "Scraped Availability: Unknown"
          return "Availability: Unknown"
      except Exception as e:
          print(f"Error occurred while scraping: {e}")
          return "Availability: Unknown"

    def search_google(self,query,num_results):
      """
      Performs a custom search  using the Google Custom Search API.

      Args:
          query (str): The search query to use.
          num_results (int, optional): The number of image search results to return (default: 3).

      Returns:
          str: A string containing information about the image search results. Each result includes the image URL and its corresponding link.
      """
      # user_query= extract_text_between_hashes(query)
      results = self.service.cse().list(
          q=query, cx=userdata.get('GOOGLE_CSE_ID'),  # Replace with your custom search engine ID
          # searchType="image",  # Specify image search
          num=3
      ).execute()

      search_results = " "
      for item in results.get("items", []):
          # for key, value in item.items():
          #   print("\033[32m" + f"{key}: {value}" + "\033[0m")

          if "pagemap" in item and "cse_image" in item["pagemap"]:
            image_url = item["pagemap"]["cse_image"][0]["src"]
            search_results += f"\n image2: {image_url} \n"
          # print(item["pagemap"]["metatags"])
          # search_results.append({"link":item["link"] ,
          #                       "title": item["title"] ,
          #                       "snippets":item["snippet"],
          #                       "image": image_url,
          #                       "meta": extract_product_info(item["pagemap"]["metatags"])})
            search_results += self.extract_product_info(item["pagemap"]["metatags"])

          search_results += f" \n link: {item['link']}"

      # len(search_results)
      return helper.clear_code(search_results)


    def combined_scraped_api_search(self, query):
        # Function to perform a combined search using both scraping and API
        scraped_results = self.google_scraper(query)
        search_api_results = self.search_google(query,3)
        return f"""{scraped_results} \n {search_api_results} """
        self.logger.info("Performing combined search for: %s", query)

# Example usage:
# scraper = WebScraper()
# query = "Apotheke 3 wick candles collection price images"

# scraper.google_scraper(query)
# scraper.combined_scraped_api_search(query)

# Create a chatbot

In [ ]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
import httpx
import asyncio
from bs4 import BeautifulSoup
import json

In [ ]:
@tool
def scrape_web(query)->str:
    """
    fetches organic search results data from Google for a given query.

    Parameters:
        query (str): The search query for which organic data is to be fetched.

    Returns:
        str: JSON formatted string containing organic search results data.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4703.0 Safari/537.36"
    }
    with httpx.Client() as client:
        response = client.get(f"https://www.google.com/search?q={query.lower().replace(' ', '+')}&gl=us&hl=en", headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        organic_results = []
        i = 0

        for el in soup.select(".g"):
            # print(el.select_one(".VwiC3b"))
            # print(type(el.select_one(".VwiC3b")))
            des = el.select_one(".VwiC3b").text if el.select_one(".VwiC3b") else ""
            title = el.select_one("h3").text if el.select_one("h3") else ""
            link = el.select_one(".yuRUbf   a")["href"] if el.select_one(".yuRUbf   a") else ""
            if title or link or des :
              organic_results.append({
                  "title":title ,
                  "link": link,
                  "description": des,
                  "rank": i+1
              })

              i+=1

        # print(organic_results)
        return json.dumps(organic_results)


# res= await get_organic_data("Apotheke 3 wick candles collection UPC ")

In [ ]:
scraper = WebScraper()
scraper.extract_availability("https://www.amazon.com/Michael-Kors-Womens-Travel-Shoulder/dp/B06W2M36C6?th=1")

'Availability: Unknown'

In [ ]:
scrape_web("Michael Kors 30S2G9HM6L 1999 amazon")

'[{"title": "Michael Kors Jet Set Travel Large Messenger Brown One ...", "link": "https://www.amazon.com/Michael-Kors-Womens-Travel-Shoulder/dp/B06W9J8LHR", "description": "Buy Michael Kors Jet Set Travel Large Messenger Brown One Size and other Messenger Bags at Amazon.com. Our wide selection is eligible for free shipping and\\u00a0...", "rank": 1}, {"title": "Michael Kors Black Medium Leather Hamilton Legacy ...", "link": "https://www.jomashop.com/michael-kors-black-medium-leather-hamilton-legacy-messenger-bag-30s2g9hm6l-001.html", "description": "Michael Kors Black Medium Leather Hamilton Legacy Messenger Bag Item No. 30S2G9HM6L-001. Authenticity Guaranteed. Since 1999, we have earned 500,000+ 5 star\\u00a0...", "rank": 2}, {"title": "Michael Kors Jet Set Large Messenger Bag Crossbody ...", "link": "https://www.amazon.com/Michael-Kors-Jet-Set/dp/B0BPMDQM2J", "description": "Sophisticated and stylish messenger bag crafted from premium PVC \\u00b7 Iconic MK signature print adds a touc

In [ ]:
@tool
def search_product(query: str) -> str:
    """
      Performs search  Google Shopping.
      Args:
          query (str): The query parameter accepts search queries with specifications such as exact-match phrases ("nikola tesla"), logical OR operators (tesla OR edison), exclusion criteria (tesla -motors), wildcard usage (tesla "rock * roll"), range matching (tesla announcement 2015..2017), price searches (tesla deposit $1000), unit conversions (250 kph in mph), searches within page titles (intitle:"tesla vs edison"), URL searches (tesla announcements inurl:2016), text searches within document bodies (intext:"orbi vs google wifi"), filetype specific searches ("tesla announcements" filetype:pdf), related site searches (related:nytimes.com), proximity searches (tesla AROUND(3) edison), and chained operator combinations ("nikola tesla" intitle:"top 5..10 facts" -site:youtube.com inurl:2015).
          Returns:
          str: A string containing information about the search query.
      """
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4703.0 Safari/537.36"
    }
    with httpx.Client() as client:
        response = client.get(f"https://www.google.com/search?sca_esv=c815caab5ae43dfb&sca_upv=1&rlz=1C1CHBF_enPK1014PK1014&sxsrf=ACQVn08_Mfup0dzdHQiTXhJJH228au87dA:1712661434013&q={query.lower().replace(' ', '+')}&tbm=shop&source=lnms&prmd=isvnbz&ved=1t:200715&ictx=111&biw=1280&bih=630&dpr=1.5", headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        body_content = soup.find('body')  # Find the body tag
        # print(body_content)
          # if body_content:
            # Extract only the text content from the body tag
        text_content = body_content.get_text(strip=True)

        # Find the element with text "Best match"
        best_match_element = soup.find(text='Best match')
        print("best_match_element: ", best_match_element)
        # Get the next sibling element
        if best_match_element:
            next_div = best_match_element.findNextSiblings('div')
            if next_div:
                # Print or do something with the content of the next div
                print("next ---------------- div---------")
                return next_div.get_text(strip=True)
            else:
               return text_content
        else:
           return text_content

        # organic_results = []
        # i = 0

        # for el in soup.select(".g"):
        #     # print(el.select_one(".VwiC3b"))
        #     # print(type(el.select_one(".VwiC3b")))
        #     des = el.select_one(".VwiC3b").text if el.select_one(".VwiC3b") else ""
        #     title = el.select_one("h3").text if el.select_one("h3") else ""
        #     link = el.select_one(".yuRUbf   a")["href"] if el.select_one(".yuRUbf   a") else ""
        #     if title or link or des :
        #       organic_results.append({
        #           "title":title ,
        #           "link": link,
        #           "description": des,
        #           "rank": i+1
        #       })

        #       i+=1

        # # print(organic_results)
        # return json.dumps(organic_results)

In [ ]:
search_product("Mineral Shower Gel Sea Kissed")

best_match_element:  Best match


<ipython-input-45-df3c4de6ab8a>:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  best_match_element = soup.find(text='Best match')


"Accessibility LinksSkip to main contentmineral shower gel sea kissedChoose what you’re giving feedback onSee moreDeleteDeleteOnly you can see this. You can delete individual items or clear your shopping history fromMy Activity.Report inappropriate predictionsSign inFilters and TopicsAllImagesShoppingVideosNewsMoreHomeOrdersSort by: RelevanceRelevanceReview scorePrice - low to highPrice - high to lowUnited StatesShow onlyOn salePriceUp to $4$4 – $7$7 – $10$10 – $25Over $25$$MinMinimum price–$$MaxMaximum priceApplyPlease provide a valid price.CategoryBody WashBody ScrubsMen's Body & Skin CareShipping & returnsFree returns1-3 day deliveryDiscoverBlack-owned businessesWomen-owned businessesProduct rating4 and up4 and up3 and up2 and up1 and upFewerMoreConditionNew itemsUsed itemsSellerAmazon.com - SellerBonanza - My Saver BoxeBayEtsyisrael catalogMercariPoshmarkAmazon.com - SellerBonanza - My Saver BoxeBayEtsyisrael catalogMercariPoshmarkSearsWalmart - UnbeatableSale LocalFewerMoreAbout t

In [ ]:
@tool
def search_web(query: str) -> str:
    """
      Performs a custom search  using the Google Custom Search API.
      Args:
          query (str): The query parameter accepts search queries with specifications such as exact-match phrases ("nikola tesla"), logical OR operators (tesla OR edison), exclusion criteria (tesla -motors), wildcard usage (tesla "rock * roll"), range matching (tesla announcement 2015..2017), price searches (tesla deposit $1000), unit conversions (250 kph in mph), searches within page titles (intitle:"tesla vs edison"), URL searches (tesla announcements inurl:2016), text searches within document bodies (intext:"orbi vs google wifi"), filetype specific searches ("tesla announcements" filetype:pdf), related site searches (related:nytimes.com), proximity searches (tesla AROUND(3) edison), and chained operator combinations ("nikola tesla" intitle:"top 5..10 facts" -site:youtube.com inurl:2015).
          Returns:
          str: A string containing information about the search query.
      """
    scraper = WebScraper()
    return scraper.search_google(query,3)

In [ ]:
@tool
def read_uploaded_file() -> str:
    """This function will return first 10 rows of uploaded file """
    sheet_id = '1nU-wu8THGDAvQjXQS8JUbueiMVISkkSa'
    sheet_name = "LevelGuide"
    processor = ExcelProcessor(sheet_id=sheet_id, sheet_name=sheet_name)
    return processor.df.head(10)

In [ ]:
tools = [ read_uploaded_file, search_web, scrape_web]

In [ ]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):

    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant. if dont find any think donot hallocinate it"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=True, memory=self.memory)

    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=1200)),
            pn.Row('WorkroomBot:', pn.pane.Markdown(self.answer, width=1190, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return

In [ ]:
cb = cbfs(tools)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [ ]:
result = cb.qa.invoke({"input": "can you read my uploaded file"})



> Entering new AgentExecutor chain...

Invoking: `read_uploaded_file` with `{}`


       Item           UPC                          Product        Size  \
0  89415031  697045164042    Mineral Shower Gel Sea Kissed   6.8 FL OZ   
1  81616031  697045163724             SuperFood Hand Cream   3.4 FL OZ   
2  89115031  697045163700  Clearing Facial Treatment Mask   2.54 FL OZ   
3  89615031  697045164769               24K Gold Mask Tube   1.7 FL OZ   
4  82013009  697045157624        DSE SALT SCRUB 220ML-ROSE      7.5 oz   
5  81313150  697045157112        DSE ALOE FOOT CREAM 5.1OZ      5.1 oz   
6  85915165  697045161249               Hydrating Mud Soap      3.4 oz   
7  84615131  697045162635        Dermud Foot Cream- no box      100 ML   
8  84215004  697045161966        Mineral hand  cream 400ml      400 ML   
9  80113005  697045160440    Body Lotion Coconut 500ml DSE      500 ML   

         SRP   Pricing off SRP       QTY   Image  Case Pack QTY  ...  \
0   $ 24.00    $ 4.00      83

In [ ]:
result = cb.qa.invoke({"input": "can you show me the first row that i gave you in sheet"})



> Entering new AgentExecutor chain...

Invoking: `read_uploaded_file` with `{}`


       Item           UPC                          Product        Size  \
0  89415031  697045164042    Mineral Shower Gel Sea Kissed   6.8 FL OZ   
1  81616031  697045163724             SuperFood Hand Cream   3.4 FL OZ   
2  89115031  697045163700  Clearing Facial Treatment Mask   2.54 FL OZ   
3  89615031  697045164769               24K Gold Mask Tube   1.7 FL OZ   
4  82013009  697045157624        DSE SALT SCRUB 220ML-ROSE      7.5 oz   
5  81313150  697045157112        DSE ALOE FOOT CREAM 5.1OZ      5.1 oz   
6  85915165  697045161249               Hydrating Mud Soap      3.4 oz   
7  84615131  697045162635        Dermud Foot Cream- no box      100 ML   
8  84215004  697045161966        Mineral hand  cream 400ml      400 ML   
9  80113005  697045160440    Body Lotion Coconut 500ml DSE      500 ML   

         SRP   Pricing off SRP       QTY   Image  Case Pack QTY  ...  \
0   $ 24.00    $ 4.00      83

In [ ]:
result = cb.qa.invoke({"input": "Take the first row from the uploaded file, sequentially fill all empty columns using both web search and scraping methods, avoiding guessing values; display the filled row at the end, marking any missing information as Not Available."})



> Entering new AgentExecutor chain...

Invoking: `read_uploaded_file` with `{}`


       Item           UPC                          Product        Size  \
0  89415031  697045164042    Mineral Shower Gel Sea Kissed   6.8 FL OZ   
1  81616031  697045163724             SuperFood Hand Cream   3.4 FL OZ   
2  89115031  697045163700  Clearing Facial Treatment Mask   2.54 FL OZ   
3  89615031  697045164769               24K Gold Mask Tube   1.7 FL OZ   
4  82013009  697045157624        DSE SALT SCRUB 220ML-ROSE      7.5 oz   
5  81313150  697045157112        DSE ALOE FOOT CREAM 5.1OZ      5.1 oz   
6  85915165  697045161249               Hydrating Mud Soap      3.4 oz   
7  84615131  697045162635        Dermud Foot Cream- no box      100 ML   
8  84215004  697045161966        Mineral hand  cream 400ml      400 ML   
9  80113005  697045160440    Body Lotion Coconut 500ml DSE      500 ML   

         SRP   Pricing off SRP       QTY   Image  Case Pack QTY  ...  \
0   $ 24.00    $ 4.00      83

In [ ]:
# result = cb.qa.invoke({"input": """Open the uploaded file containing the rows to process.
# For each row in the file:
# Check if there are any empty columns.
# If there are empty columns:
# Use web search to fill in the missing information.
# If the required information is not found through web search, proceed to scraping the data.
# If the information cannot be found through scraping either, mark it as "Not Available."
# Once all empty columns are filled or marked as "Not Available" for each row, display the results.
# Make sure to handle each row sequentially, filling in missing information as accurately as possible and avoiding guessing values. """})

In [ ]:
# result = cb.qa.invoke({"input": "what is image url can you write origal url of image"})


In [ ]:

# result = cb.qa.invoke({"input": "can you update the data with all the information you collected"})


In [ ]:


inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    # pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400, width=1300),
    pn.layout.Divider(),
    pn.Row(inp),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Workroom Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Divider()
            [1] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True, sizing_mode='fixed', width=1300)
            [2] Divider()
            [3] Row
                [0] TextInput(placeholder='Enter text here…')